### Functions and Objects

In [54]:
import yaml
import pandas as pd
from web3 import Web3
import datetime
import numpy as np
import json
import os

# Function to read in YAML file
def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)
    
# read in all yaml files
token_addresses = read_yaml_file('token-addresses.yml')
ethereum_token_addresses = token_addresses['ethereum']
track_bridged_eth = read_yaml_file('track-bridged-eth.yml')

# Connect to Ethereum node
rpc_url = 'https://eth.llamarpc.com'
rpc_url = 'https://ethereum-rpc.publicnode.com'
rpc_url = 'https://eth-mainnet.public.blastapi.io'
rpc_url = 'https://mainnet.gateway.tenderly.co'
w3 = Web3(Web3.HTTPProvider(rpc_url))

# Function to get Ethereum balance for each address
def get_eth_balance(w3: Web3, address, at_block='latest'):
    try:
        balance = w3.eth.get_balance(Web3.to_checksum_address(address), block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving balance for {address}: {e}")
    return balance

# Function to get ERC20 balance of a given coin & address
def get_erc20_balance_ethereum(w3: Web3, coin, address, at_block='latest'):
    contract_address = ethereum_token_addresses[coin]['contract']
    ABI = ethereum_token_addresses[coin]['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(contract_address), abi=ABI)
        balance = contract.functions.balanceOf(Web3.to_checksum_address(address)).call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving balance for {address}: {e}")
    return balance

# Function to get prices of wstETH, we assume 1stETH = 1 ETH
def get_ETH_wstETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['wstETH']['contract']
    abi = ethereum_token_addresses['wstETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.stEthPerToken().call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for wstETH: {e}")
    return price

# Function to get price of mETH
def get_ETH_mETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['mETH']['staking_contract']
    abi = ethereum_token_addresses['mETH']['staking_abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.mETHToETH(10**18).call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for mETH: {e}")
    return price

# Function to get price of pufETH
def get_ETH_pufETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['pufETH']['contract']
    abi = ethereum_token_addresses['pufETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.convertToAssets(10**18).call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for pufETH: {e}")
    return price

# Function to get price of weETH, assuming 1 eETH = 1 ETH
def get_ETH_weETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['weETH']['contract']
    abi = ethereum_token_addresses['weETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.getEETHByWeETH(10**18).call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for weETH: {e}")
    return price

# Function to get price of rETH
def get_ETH_rETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['rETH']['contract']
    abi = ethereum_token_addresses['rETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.getExchangeRate().call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for rETH: {e}")
    return price

# Function to get price of cbETH
def get_ETH_cbETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['cbETH']['contract']
    abi = ethereum_token_addresses['cbETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.exchangeRate().call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for cbETH: {e}")
    return price

# Function to get price of sfrxETH
def get_ETH_sfrxETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['sfrxETH']['contract']
    abi = ethereum_token_addresses['sfrxETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.pricePerShare().call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for sfrxETH: {e}")
    return price

# Function to get price of rswETH 
def get_ETH_rswETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['rswETH']['contract']
    abi = ethereum_token_addresses['rswETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.rswETHToETHRate().call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for rswETH: {e}")
    return price

# Function to get price of rsETH 
def get_ETH_rsETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['rsETH']['deposit_contract']
    abi = ethereum_token_addresses['rsETH']['deposit_abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = 10**18/contract.functions.getRsETHAmountToMint(ethereum_token_addresses['stETH']['contract'], 10**18).call(block_identifier=at_block)
    except Exception as e:
        print(f"Error retrieving price for rsETH: {e}")
    return price

# Function to get price of ezETH 
def get_ETH_ezETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['ezETH']['contract']
    abi = ethereum_token_addresses['ezETH']['abi']
    deposit_address = ethereum_token_addresses['ezETH']['deposit_contract']
    deposit_abi = ethereum_token_addresses['ezETH']['deposit_abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        deposit_contract = w3.eth.contract(address=Web3.to_checksum_address(deposit_address), abi=deposit_abi)
        total_supply_ezETH = contract.functions.totalSupply().call(block_identifier=at_block)
        total_eth = deposit_contract.functions.calculateTVLs().call(block_identifier=at_block)[2]
        price = total_eth / total_supply_ezETH
    except Exception as e:
        print(f"Error retrieving price for ezETH: {e}")
    return price

# Function to get the block number of the first block of the day using binary search
def get_first_block_of_day(w3: Web3, target_date: datetime.date):
    # Convert the target date to a UNIX timestamp (00:00:00 of that day in UTC)
    start_of_day = datetime.datetime.combine(target_date, datetime.time(0, 0), tzinfo=datetime.timezone.utc)
    start_timestamp = int(start_of_day.timestamp())

    # Get the current block number (latest block)
    latest_block = w3.eth.get_block('latest')['number']

    # Perform binary search to find the block with the timestamp >= start_timestamp
    low = 0
    high = latest_block

    while low < high:
        mid = (low + high) // 2
        mid_block = w3.eth.get_block(mid)
        if mid_block['timestamp'] < start_timestamp:
            low = mid + 1
        else:
            high = mid

    # After the search, 'low' should be the first block of the day or a block at or after the target timestamp
    first_block_of_day = w3.eth.get_block(low)

    return first_block_of_day if first_block_of_day['timestamp'] >= start_timestamp else None

# Function to lookup the block number of a given date using block_timestamps.csv
def lookup_block_number(date: str):
    df = pd.read_csv('block_timestamps.csv')
    try:
        block_number = int(df.loc[df['date'] == date, 'block'].values[0])
    except:
        block_number = None
    return block_number

## Script to backfill block_timestamps.csv

In [55]:
# get the last date in the csv
df = pd.read_csv('block_timestamps.csv')
last_date = pd.to_datetime(df['date'].iloc[-1]).date()

# get current date
current_date = datetime.datetime.now().date()

while current_date > last_date:
    # get block number
    new_block = get_first_block_of_day(w3, last_date + datetime.timedelta(days=1))
    new_row = pd.DataFrame({'date': [str(last_date + datetime.timedelta(days=1))], 
                            'block': [new_block['number']], 
                            'block_timestamp': [new_block['timestamp']]})
    df = pd.concat([df, new_row], ignore_index=True)
    last_date = last_date + datetime.timedelta(days=1)
    # save to csv
    df.to_csv('block_timestamps.csv', index=False)

## Script to create folder structure and empty csv files

In [56]:
# get an empty dataframe
df = pd.read_csv('block_timestamps.csv')
df = df.drop(columns=['block', 'block_timestamp'])

# read in all yaml files
token_addresses = read_yaml_file('token-addresses.yml')
ethereum_token_addresses = token_addresses['ethereum']
track_bridged_eth = read_yaml_file('track-bridged-eth.yml')

# create holdings folder
if os.path.exists('holdings') == False:
    os.mkdir('holdings')

# create a dictionary for each entity
for entity in list(track_bridged_eth):
    if os.path.exists(f"holdings/{entity}") == False:
        os.mkdir(f"holdings/{entity}")

# create subfolder for each chain in entity
for entity in list(track_bridged_eth):
    for chain in list(track_bridged_eth[entity]):
        if os.path.exists(f"holdings/{entity}/{chain}") == False:
            os.mkdir(f"holdings/{entity}/{chain}")

# create a csv file for each token in each chain
for entity in list(track_bridged_eth):
    for chain in list(track_bridged_eth[entity]):
        for token in [list(item.keys())[0] for item in track_bridged_eth[entity][chain]]:
            if os.path.exists(f"holdings/{entity}/{chain}/{token}.csv") == False:
                df.to_csv(f"holdings/{entity}/{chain}/{token}.csv", index=False)

# create prices folder
if os.path.exists('prices') == False:
    os.mkdir('prices')

# create a folder for each chain in prices folder
for chain in list(token_addresses.keys()):
    if os.path.exists(f"prices/{chain}") == False:
        os.mkdir(f"prices/{chain}")

# create a csv file for each token in each chain
for chain in list(token_addresses.keys()):
    for token in list(token_addresses[chain]):
        if os.path.exists(f"prices/{chain}/{token}.csv") == False:
            df.to_csv(f"prices/{chain}/{token}.csv", index=False)


### extend the csv files with the new dates

for entity in list(track_bridged_eth):
    for chain in list(track_bridged_eth[entity]):
        for token in [list(item.keys())[0] for item in track_bridged_eth[entity][chain]]:
            df = pd.read_csv(f"holdings/{entity}/{chain}/{token}.csv")
            last_date = pd.to_datetime(df['date'].iloc[-1]).date()
            column_names = df.columns
            for date in pd.date_range(start=last_date + datetime.timedelta(days=1), end=current_date):
                new_row = pd.DataFrame({'date': [date.strftime('%Y-%m-%d')]}, columns=column_names)
                new_row.to_csv(f"holdings/{entity}/{chain}/{token}.csv", index=False, mode='a', header=False)

for chain in list(token_addresses.keys()):
    for token in list(token_addresses[chain]):
        df = pd.read_csv(f"prices/{chain}/{token}.csv")
        last_date = pd.to_datetime(df['date'].iloc[-1]).date()
        column_names = df.columns
        for date in pd.date_range(start=last_date + datetime.timedelta(days=1), end=current_date):
            new_row = pd.DataFrame({'date': [date.strftime('%Y-%m-%d')]}, columns=column_names)
            new_row.to_csv(f"prices/{chain}/{token}.csv", index=False, mode='a', header=False)

## Script to backfill holdings on Ethereum

In [57]:
# read in all yaml files
token_addresses = read_yaml_file('token-addresses.yml')
ethereum_token_addresses = token_addresses['ethereum']
track_bridged_eth = read_yaml_file('track-bridged-eth.yml')

for entity in track_bridged_eth.keys():

    print(f"Processing {entity}")

    for j in track_bridged_eth[entity]['ethereum']:

        # create a column for each address if not already in the dataframe
        asset = list(j.keys())[0]
        addresses = j[asset]
        df = pd.read_csv(f'holdings/{entity}/ethereum/{asset}.csv')
        for address in addresses[0]['address']:
            if address not in df.columns:
                df[address] = np.nan

        for address in addresses[0]['address']:
            # only process if the last row of the column is not yet filled
            if np.isnan(df[address].iloc[-1]) == False:
                print(f"Skipping {address} as already filled")
                pass
            else:
                print(f"Backfilling {address}")
                # iterate through each row from reverse until we get balance = 0
                for i in range(len(df)-1, -1, -1):
                    date = df['date'].iloc[i]
                    block = lookup_block_number(date)
                    #print(f"Processing {date}")
                    if asset == 'ETH':
                        balance = get_eth_balance(w3, address, block)
                    else:
                        balance = get_erc20_balance_ethereum(w3, asset, address, block)

                    if np.isnan(df.loc[i, address]):
                        df.loc[i, address] = balance
                        if i % 100 == 0:
                            df.to_csv(f"holdings/{entity}/ethereum/{asset}.csv", index=False)
                    else: # stop if we already have a value (full backfilled)
                        break

                    # stop also if balance is 0 = contract not yet used or deployed / no coins on it
                    #if balance == 0:
                    #    break

                # save to csv
                df.to_csv(f"holdings/{entity}/ethereum/{asset}.csv", index=False)


Processing Beacon_chain_deposits
Backfilling 0x00000000219ab540356cbb839cbe05303d7705fa
Processing Arbitrum
Backfilling 0x8315177ab297ba92a06054ce80a67ed4dbd7ed3a
Backfilling 0x011B6E24FfB0B5f5fCc564cf4183C5BBBc96D515
Backfilling 0x0F25c1DC2a9922304f2eac71DCa9B07E310e8E5a
Processing Base
Backfilling 0x49048044d57e1c92a77f79988d21fa8faf74e97e
Backfilling 0x9de443AdC5A411E83F1878Ef24C3F52C61571e72
Backfilling 0x3154Cf16ccdb4C6d922629664174b904d80F2C35
Processing Optimism
Backfilling 0xbEb5Fc579115071764c7423A4f12eDde41f106Ed
Backfilling 0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1
Backfilling 0x76943C0D61395d8F2edF9060e1533529cAe05dE6
Processing Scroll
Backfilling 0x6774Bcbd5ceCeF1336b5300fb5186a12DDD8b367
Backfilling 0xA033Ff09f2da45f0e9ae495f525363722Df42b2a
Backfilling 0x6625C6332c9F91F2D27c304E729B86db87A3f504
Processing Blast
Backfilling 0x98078db053902644191f93988341E31289E1C8FE
Backfilling 0x5F6AE08B8AeB7078cf2F96AFb089D7c9f51DA47d
Backfilling 0x98078db053902644191f93988341E31289E1C

## Script to backfill prices

In [58]:
# only works for assets on ethereum L1 as of now!

for chain in token_addresses:

    print(f"Processing {chain}")

    for token in token_addresses[chain]:

        # create a column for each address if not already in the dataframe
        df = pd.read_csv(f'prices/{chain}/{token}.csv')
        if 'price' not in df.columns:
            df['price'] = None
            df.to_csv(f"prices/{chain}/{token}.csv", index=False)

        # only process if the whole column is not yet filled
        if np.isnan(df['price'].iloc[-1]) == False:
            print(f"Skipping {token} as already filled")
            pass
        else:
            print(f"Backfilling {token}")
            # iterate through each row from reverse until we get price = 0
            for i in range(len(df)-1, -1, -1):
                date = df['date'].iloc[i]
                block = lookup_block_number(date)
                try:
                    if token == 'wstETH':
                        price = get_ETH_wstETH_price(w3, block)
                    elif token == 'mETH':
                        price = get_ETH_mETH_price(w3, block)
                    elif token == 'pufETH':
                        price = get_ETH_pufETH_price(w3, block)
                    elif token == 'weETH':
                        price = get_ETH_weETH_price(w3, block)
                    elif token == 'rETH':
                        price = get_ETH_rETH_price(w3, block)
                    elif token == 'cbETH':
                        price = get_ETH_cbETH_price(w3, block)
                    elif token == 'sfrxETH':
                        price = get_ETH_sfrxETH_price(w3, block)
                    elif token == 'rswETH':
                        price = get_ETH_rswETH_price(w3, block)
                    elif token == 'rsETH':
                        price = get_ETH_rsETH_price(w3, block)
                    elif token == 'ezETH':
                        price = get_ETH_ezETH_price(w3, block)
                    elif token in ['wETH', 'stETH', 'eETH', 'pxETH', 'frxETH']: # assuming 1 token = 1 ETH
                        price = 1
                    else:
                        print(f"Token {token} not supported")
                except Exception as e:
                    print(f"Error retrieving price for {token}: {e}")
                    print("Assuming date reached where contract not yet deployed or implementation changed.")
                    break
                df.loc[i, 'price'] = price

                # break the loop if value is filled
                if np.isnan(df.loc[i-1, 'price']) == False:
                    break

                # stop if balance is 0 = contract not yet used
                if price == 0:
                    break

            # save to csv
            df.to_csv(f"prices/{chain}/{token}.csv", index=False)

Processing ethereum
Backfilling wETH
Backfilling stETH
Backfilling wstETH
Backfilling mETH
Backfilling cbETH
Backfilling pufETH
Backfilling eETH
Backfilling weETH
Backfilling rETH
Backfilling ezETH
Backfilling frxETH
Backfilling sfrxETH
Backfilling pxETH
Backfilling rswETH
Backfilling rsETH


## Script for Bitcoin RPC endpoint

In [24]:
# this script get the current block number and then calculates the total supply of BTC based on the issuance schedule
import requests
import json

btc_rpc = 'https://rpc.ankr.com/btc'

def bitcoin_rpc_call(method, params=[]):
    headers = {'content-type': 'application/json'}

    # No authentication required for public rpcs
    payload = json.dumps({
        "method": method,
        "params": params,
        "jsonrpc": "2.0",
        "id": 0,
    })

    try:
        response = requests.post(btc_rpc, headers=headers, data=payload)
        response.raise_for_status()  # Check for HTTP request errors
        return response.json().get('result')
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
    
def calculate_btc_supply(block_height):
    total_supply = 0 # at block 0
    reward = 50  # Initial block reward
    halving_interval = 210000

    # Loop over halving periods
    i = 0
    while block_height > 0:
        blocks_in_current_period = min(block_height, halving_interval)
        total_supply += blocks_in_current_period * reward
        block_height -= blocks_in_current_period
        reward /= 2  # Halve the reward
        i += 1

    return total_supply

# full list of rpc endpoints for bitcoin: https://developer.bitcoin.org/reference/rpc/
latest_block_count = bitcoin_rpc_call("getblockcount")
total_btc = calculate_btc_supply(latest_block_count)
print(f"Total BTC supply: {total_btc}")

# TODO: write a script that gets the first bitcoin block for each day
# TODO: then backfill the supply of BTC at each block using calculate_btc_supply (offline compute)

Total BTC supply: 19774193.75


## Gold Total Supply

In [61]:
# we base this on historic yearly data (above ground stock) & linear interpolate number between know data points
# in case the date is in the future we project the growth rate of the last year into the future linearly 
# source which is updatede yearly: https://www.gold.org/goldhub/data/how-much-gold#from-login=1&just-verified=1

data = {
    "2010": 168245.7,
    "2011": 171145.0,
    "2012": 174056.9,
    "2013": 177195.8,
    "2014": 180571.2,
    "2015": 183945.4,
    "2016": 187498.2,
    "2017": 191048.4,
    "2018": 194692.7,
    "2019": 198295.2,
    "2020": 201738.2,
    "2021": 205309.3,
    "2022": 208921.0,
    "2023": 212582.5
}

def get_gold_above_ground_supply(date: datetime, data: dict):

    # Convert years to ints for easier comparison
    data_years = {int(k): v for k, v in data.items()}
    year = date.year
    
    # Calculate day progress (0 to 1)
    days_in_year = 366 if year % 4 == 0 else 365
    progress = (date.timetuple().tm_yday - 1) / days_in_year
    
    # Handle dates within data range
    if year in data_years:
        if progress == 1.0:
            return data_years[year]
            
        next_year = year + 1
        if next_year in data_years:
            # Simple linear interpolation between years
            return round(data_years[year] + (data_years[next_year] - data_years[year]) * progress, 1)
    
    # Handle future dates
    max_year = max(data_years.keys())
    if year > max_year:
        # Calculate the yearly growth rate using the last year's data
        last_year_growth = data_years[max_year] - data_years[max_year - 1]
        years_difference = year - max_year
        days_progress = progress
        
        # Project into the future using the growth rate
        projected_value = data_years[max_year] + (last_year_growth * years_difference) + (last_year_growth * days_progress)
        return round(projected_value, 1)
        
    raise ValueError(f"Date must be after {min(data_years.keys())}")



In [64]:
from datetime import datetime
# usage example
#get_gold_above_ground_supply(datetime.now(), data) 
get_gold_above_ground_supply(datetime(2024, 1, 1), data)

216244.0

## Script to get the ETH total supply over time ...? tba via Dune query

In [ ]:
# Etherscan is paid plan only
# Coingecko API is free but only estimate
# DUNE???
# TODO!

import requests

# Etherscan API key
ETHERSCAN_API_KEY = os.environ['ETHERSCAN_API_KEY']

# Function to get ETH total supply at a block
def get_eth_supply_at_block(block_number):
    url = f'https://api.etherscan.io/api?module=proxy&action=eth_getBlockByNumber&tag={hex(block_number)}&boolean=true&apikey={ETHERSCAN_API_KEY}'
    response = requests.get(url)
    data = response.json()

    if 'result' in data:
        block_reward = int(data['result']['blockReward'], 16)  # Block reward in Wei
        return block_reward / (10 ** 18)  # Convert Wei to ETH
    else:
        print(f"Error fetching ETH supply at block {block_number}: {data}")
        return None
    


## Get Beaconchain Staked Amount Per Validator Set

In [26]:
# read in octant_validators.txt
with open('octant_validators.txt', 'r') as file:
    octant_validators = file.read().splitlines()

# turn str to number
octant_validators = [int(i) for i in octant_validators]

In [38]:
import requests

def get_validator_balance(validator_ids, slot='head'):
    url = f'https://ethereum-beacon-api.publicnode.com/eth/v1/beacon/states/{slot}/validators'
    total_balance = 0
    all_data = []

    for i in range(0, len(validator_ids), 100):
        batch = validator_ids[i:i+100]
        params = {'id': ','.join(map(str, batch))}
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            # Extract and sum up balances
            balances = [int(validator['balance']) for validator in data['data']]
            total_balance += sum(balances)
            all_data.extend(data['data'])
        except requests.exceptions.RequestException as e:
            print(f"Error fetching validator rewards: {e}")
            return None, 0

    return all_data, total_balance

# Example usage
result, total_balance = get_validator_balance(octant_validators)
print("Validators:", len(result))
print("Total Balance:", total_balance/10**9, "ETH")


Validators: 3136
Total Balance: 100417.20743824 ETH


## Get POS Rewards & Slashing per slot

In [46]:
import requests

rpc_endpoint = "https://lb.drpc.org/rest/eth-beacon-chain"
rpc_endpoint2 = "https://ethereum-beacon-api.publicnode.com"

def get_beacon_rewards(slot="head"):
    url = f'{rpc_endpoint3}/eth/v1/beacon/rewards/blocks/{slot}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching rewards for slot {slot}: {e}")
        return None

# Example usage
slot = 10476864
rewards = get_beacon_rewards(slot)
print(rewards)

# problem: old slots take a lot of time to fetch...? 60s+

{'execution_optimistic': False, 'finalized': True, 'data': {'proposer_index': '210278', 'total': '46021514', 'attestations': '44398130', 'sync_aggregate': '1623384', 'proposer_slashings': '0', 'attester_slashings': '0'}}


In [54]:
df_all = pd.DataFrame()  # Initialize df_all as an empty DataFrame

for i in range(0, 10):
    slot = 10476864 + i
    rewards = get_beacon_rewards(slot)
    rewards['data']['slot'] = slot
    df = pd.DataFrame([rewards['data']])
    df_all = pd.concat([df_all, df], ignore_index=True)

df_all.to_csv('beacon_rewards.csv', index=False)